<a href="https://colab.research.google.com/github/monicafar147/classification-predict-streamlit-template/blob/Modeling/trying_various_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install comet_ml

In [1]:
# import comet_ml in the top of your file
from comet_ml import Experiment
    
# Add the following code anywhere in your machine learning file
experiment = Experiment(api_key="rBqQ3hDuEa6xVpT9ns5Tz1dVt",project_name="nlp-climate-change", workspace="monicafar147")

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/monicafar147/nlp-climate-change/ce2af673f20142018236c705c1bc7a88



In [2]:
import numpy as np 
import pandas as pd

# plotting
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-deep')

# text preprocessing
import re
from string import punctuation
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from textblob import Word
from wordcloud import WordCloud, STOPWORDS

# models
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

nltk.download('stopwords')
nltk.download('punkt')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
train = pd.read_csv("https://raw.githubusercontent.com/monicafar147/classification-predict-streamlit-template/master/climate-change-belief-analysis/train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/monicafar147/classification-predict-streamlit-template/master/climate-change-belief-analysis/test.csv")

In [4]:
print("Train\n")
print(train.head(5))
print("\nTest")
print(test.head(5))

Train

   sentiment                                            message  tweetid
0          1  PolySciMajor EPA chief doesn't think carbon di...   625221
1          1  It's not like we lack evidence of anthropogeni...   126103
2          2  RT @RawStory: Researchers say we have three ye...   698562
3          1  #TodayinMaker# WIRED : 2016 was a pivotal year...   573736
4          1  RT @SoyNovioDeTodas: It's 2016, and a racist, ...   466954

Test
                                             message  tweetid
0  Europe will now be looking to China to make su...   169760
1  Combine this with the polling of staffers re c...    35326
2  The scary, unimpeachable evidence that climate...   224985
3  @Karoli @morgfair @OsborneInk @dailykos \nPuti...   476263
4  RT @FakeWillMoore: 'Female orgasms cause globa...   872928


In [4]:
def preprocess(tweet):
  tweet = tweet.lower()
  tweet = re.sub(r"\W", " ", tweet)
  tweet = re.sub(r'#([^\s]+)', r'\1', tweet) 
  tweet = word_tokenize(tweet)
  stopwords_list = set(stopwords.words('english') + list(punctuation))
  tweets = [word for word in tweet if word not in stopwords_list]
  return " ".join(tweet)

In [5]:
# Splitting the labels and features
train['processed'] = train['message'].apply(preprocess)
X = train['processed']
y = train['sentiment']

In [6]:
from sklearn.model_selection import train_test_split

# Splitting the labels and fetures into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,random_state=42)

In [8]:
# Decision tree
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import tree

clf = tree.DecisionTreeClassifier()
text_cls = Pipeline([('tfidf',TfidfVectorizer()),('classify',clf)])
text_cls.fit(X_train, y_train)

pred = text_cls.predict(X_test)
from sklearn.metrics import confusion_matrix,classification_report
report = print(classification_report(y_test, pred))

              precision    recall  f1-score   support

          -1       0.28      0.21      0.24       126
           0       0.35      0.34      0.35       224
           1       0.70      0.72      0.71       895
           2       0.56      0.58      0.57       337

    accuracy                           0.60      1582
   macro avg       0.47      0.47      0.47      1582
weighted avg       0.59      0.60      0.59      1582



In [9]:
# Create dictionaries for the data we want to log

params = {"preprocessing":  "preprocess(data)",
          "model_type": "DecisionTreeClassifier",
          "test size":"0.1"
          }

metrics = {"report" : report,
           }

In [10]:
# Linear SVC
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

svc = LinearSVC()
text_svc = Pipeline([('tfidf',TfidfVectorizer()),('classify',svc)])
text_svc.fit(X_train, y_train)

pred_2 = text_svc.predict(X_test)
from sklearn.metrics import confusion_matrix,classification_report
report_2 = print(classification_report(y_test, pred_2))
report_2

              precision    recall  f1-score   support

          -1       0.67      0.52      0.59       126
           0       0.66      0.42      0.52       224
           1       0.80      0.86      0.83       895
           2       0.75      0.83      0.79       337

    accuracy                           0.77      1582
   macro avg       0.72      0.66      0.68      1582
weighted avg       0.76      0.77      0.76      1582



In [11]:
# Create dictionaries for the data we want to log

params = {"preprocessing":  "preprocess(data)",
          "model_type": "LinearSVC",
          }

metrics = {"report" : report_2,
           }

In [ ]:
experiment.end()

In [ ]:
# Gaussian not working though
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
naive_bayes = GaussianNB()
text_nb = Pipeline([('tfidf',TfidfVectorizer()),('classify',naive_bayes)])
XD_train = X_train.toarray()
text_nb.fit(XD_train, y_train)

pred_3 = text_nb.predict(X_test)
report_3 = print(classification_report(y_test, pred_3))
report_3


In [10]:
# K nearest neighbours, 250 seems best but maybe could be a better option
n_neighbors = 250
from sklearn.neighbors import KNeighborsClassifier 
knn = KNeighborsClassifier(n_neighbors)
text_knn = Pipeline([('tfidf',TfidfVectorizer()),('classify',knn)])
text_knn.fit(X_train, y_train)

pred_4 = text_knn.predict(X_test)
from sklearn.metrics import confusion_matrix,classification_report
report_4 = print(classification_report(y_test, pred_4))
report_4



              precision    recall  f1-score   support

          -1       1.00      0.01      0.02       126
           0       0.88      0.03      0.06       224
           1       0.63      0.97      0.76       895
           2       0.84      0.45      0.59       337

    accuracy                           0.65      1582
   macro avg       0.83      0.37      0.36      1582
weighted avg       0.74      0.65      0.57      1582



In [11]:
params = {"preprocessing":  "_preprocess(data)",
          "model_type": "KNeighborsClassifier",
          }

metrics = {"report" : report_4,
           }

In [12]:
# Log our parameters and results
experiment.log_parameters(params)
experiment.log_metric("report",report_4)

In [13]:
experiment.end()

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/monicafar147/nlp-climate-change/9eaa9b8e6ab7479ab682ebc63d3fb16d
COMET INFO:   Metrics:
COMET INFO:     report : 1
COMET INFO:   Parameters:
COMET INFO:     classify_algorithm     : auto
COMET INFO:     classify_leaf_size     : 30
COMET INFO:     classify_metric        : minkowski
COMET INFO:     classify_metric_params : 1
COMET INFO:     classify_n_jobs        : 1
COMET INFO:     classify_n_neighbors   : 250
COMET INFO:     classify_p             : 2
COMET INFO:     classify_weights       : uniform
COMET INFO:     model_type             : KNeighborsClassifier
COMET INFO:     preprocessing          : _preprocess(data)
COMET INFO:     tfidf_analyzer         : word
COMET INFO:     tfidf_binary           : 1
COMET INFO:     tfidf_decode_error     : s

In [9]:
# MulitnomialNB
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
text_mnb = Pipeline([('tfidf',TfidfVectorizer()),('classify',model)])
text_mnb.fit(X_train, y_train)
pred_5 = text_mnb.predict(X_test)
from sklearn.metrics import confusion_matrix,classification_report
report_5 = print(classification_report(y_test, pred_5))
report_5


              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       126
           0       1.00      0.03      0.05       224
           1       0.61      0.99      0.76       895
           2       0.93      0.33      0.49       337

    accuracy                           0.64      1582
   macro avg       0.63      0.34      0.32      1582
weighted avg       0.68      0.64      0.54      1582



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
params = {"preprocessing":  "_preprocess(data)",
          "model_type": "MultinomialNB",
          }

metrics = {"report" : report_5,
           }

In [11]:
# Log our parameters and results
experiment.log_parameters(params)
experiment.log_metric("report",report_5)

In [12]:
experiment.end()

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/monicafar147/nlp-climate-change/0123d48d967841669705fa90ac40a214
COMET INFO:   Metrics:
COMET INFO:     report : 1
COMET INFO:   Parameters:
COMET INFO:     alpha                : 1.0
COMET INFO:     class_prior          : 1
COMET INFO:     classify_alpha       : 1.0
COMET INFO:     classify_class_prior : 1
COMET INFO:     classify_fit_prior   : True
COMET INFO:     fit_prior            : True
COMET INFO:     model_type           : MultinomialNB
COMET INFO:     preprocessing        : _preprocess(data)
COMET INFO:     tfidf_analyzer       : word
COMET INFO:     tfidf_binary         : 1
COMET INFO:     tfidf_decode_error   : strict
COMET INFO:     tfidf_dtype          : <class 'numpy.float64'>
COMET INFO:     tfidf_encoding       : utf-8
COMET INFO:

In [7]:
# Logistic regression
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
text_lr = Pipeline([('tfidf',TfidfVectorizer()),('classify',lr)])
text_lr.fit(X_train, y_train)
pred_6 = text_lr.predict(X_test)
from sklearn.metrics import confusion_matrix,classification_report
report_6 = print(classification_report(y_test, pred_6))
report_6

              precision    recall  f1-score   support

          -1       0.75      0.31      0.44       126
           0       0.67      0.34      0.45       224
           1       0.76      0.88      0.82       895
           2       0.74      0.80      0.77       337

    accuracy                           0.75      1582
   macro avg       0.73      0.59      0.62      1582
weighted avg       0.74      0.75      0.72      1582



sklearn.linear_model._logistic:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [8]:
params = {"preprocessing":  "_preprocess(data)",
          "model_type": "LogisticRegression",
          }

metrics = {"report" : report_6,
           }

In [9]:
# Log our parameters and results
experiment.log_parameters(params)
experiment.log_metric("report",report_6)

In [10]:
experiment.end()

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/monicafar147/nlp-climate-change/ce2af673f20142018236c705c1bc7a88
COMET INFO:   Metrics:
COMET INFO:     report : 1
COMET INFO:   Parameters:
COMET INFO:     C                          : 1.0
COMET INFO:     class_weight               : 1
COMET INFO:     classify_C                 : 1.0
COMET INFO:     classify_class_weight      : 1
COMET INFO:     classify_dual              : 1
COMET INFO:     classify_fit_intercept     : True
COMET INFO:     classify_intercept_scaling : 1
COMET INFO:     classify_l1_ratio          : 1
COMET INFO:     classify_max_iter          : 100
COMET INFO:     classify_multi_class       : auto
COMET INFO:     classify_n_jobs            : 1
COMET INFO:     classify_penalty           : l2
COMET INFO:     classify_random_state  